In [1]:
import numpy as np
import pandas as pd
import os, sys
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [16]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## Reading data

In [2]:
jan_trip = pd.read_parquet('yellow_tripdata_2022-01.parquet')
feb_trip = pd.read_parquet('yellow_tripdata_2022-02.parquet')

In [3]:
jan_trip.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [4]:
print(jan_trip.shape)

(2463931, 19)


In [5]:
print(feb_trip.shape)

(2979431, 19)


In [6]:
#Data Transformations

#Converting to datetime
jan_trip['tpep_dropoff_datetime'] = pd.to_datetime( jan_trip['tpep_dropoff_datetime'] )
jan_trip['tpep_pickup_datetime']  = pd.to_datetime( jan_trip['tpep_pickup_datetime'] )

feb_trip['tpep_dropoff_datetime'] = pd.to_datetime( feb_trip['tpep_dropoff_datetime'] )
feb_trip['tpep_pickup_datetime']  = pd.to_datetime( feb_trip['tpep_pickup_datetime'] )

In [7]:
jan_trip['duration'] = jan_trip.tpep_dropoff_datetime - jan_trip.tpep_pickup_datetime
jan_trip.duration = jan_trip.duration.apply(lambda td: td.total_seconds() / 60)

In [8]:
jan_trip.duration.std()

46.44530513776499

In [9]:
jan_trip[(jan_trip.duration >= 1) & (jan_trip.duration <= 60)].shape[0] / jan_trip.shape[0]

0.9827547930522406

In [10]:
jan_trip = jan_trip[(jan_trip.duration >= 1) & (jan_trip.duration <= 60)]

In [11]:
jan_trip.shape

(2421440, 20)

In [13]:
categorical = ['PULocationID', 'DOLocationID']
train_dicts = jan_trip[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = jan_trip[target].values

In [14]:
X_train.shape

(2421440, 2)

# Fitting Model

In [17]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

8.920327827581444

## Data Transformation Pipeline

In [18]:
def data_pipeline(df):
    df['tpep_dropoff_datetime'] = pd.to_datetime( df['tpep_dropoff_datetime'] )
    df['tpep_pickup_datetime']  = pd.to_datetime( df['tpep_pickup_datetime'] )
    
    #creating duration columns
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    
    #filter between 1 to 60 minutes
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    return df

In [19]:
def create_x_and_y(df):
    categorical = ['PULocationID', 'DOLocationID']
    dicts = df[categorical].to_dict(orient='records')

    dv = DictVectorizer()
    X = dv.fit_transform(dicts)

    target = 'duration'
    y = df[target].values
    
    return X , y

## Fitting model on Feb data

In [20]:
X_val, y_val = create_x_and_y( data_pipeline(feb_trip) )
y_val_pred = lr.predict(X_val)

mean_squared_error(y_val, y_val_pred, squared=False)

9.638272212087234